[View in Colaboratory](https://colab.research.google.com/github/3to80/DeepLearning/blob/master/MNIST_CNN.ipynb)

In [0]:
# 파이썬 2와 파이썬 3 지원
from __future__ import division, print_function, unicode_literals

# 공통
import numpy as np
import os
import tensorflow as tf
import sklearn

# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# 맷플롯립 설정
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
plt.rcParams['font.family'] = 'NanumBarunGothic'
plt.rcParams['axes.unicode_minus'] = False

# 그림을 저장할 폴더
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [0]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image.astype(np.uint8),interpolation="nearest")
    plt.axis("off")

###Build graph
 
 - **mnist** : 28 * 28 * 1

- **cnn** 
  - conv : 64, [3, 3], 1 SAME
  - pool1: 64, [3, 3], 2 SAME
  - conv: 128, [3, 3], 1 SAME
  - pool2: 128, [3,3], 2 SAME
  - flat : tf.reshape(pool2, shape=[-1, conv2_fmaps * w/4 * h/4])
  
  
 - **dnn** 
  - dropout : 0.5 //  bn_momentum : 0.95
  - layer1 : 128
    - drop1
    - fc1
    - bn1
    
  - layer2 : 128
    - drop2
    - fc2
    - bn2

In [87]:
height = 28
width = 28


channels = 1
n_inputs = height * width

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"


pool1_fmaps = 32
pool1_stride = 2
pool1_pad = "SAME"
# 14 14 


conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"
#14 14

n_outputs = 10

reset_graph()
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)



with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels], name="X_reshape")
    y = tf.placeholder(tf.int32, shape=[None], name="y")


## X_input : 128, 784, reshape : X_reshape : 128, 28, 28, 1 

with tf.name_scope("cnn"):    
  # X_reshape : NONE, 28, 28, 1
  print("인풋 shape " , X_reshaped.shape)
  
  conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.selu, name="conv1")
  # 28 28 conv1_fmaps
  # NONE , 28 , 28 , 32 
  print("conv1 shape " , conv1.shape)

  
  pool1 = tf.nn.max_pool(conv1, ksize = [1,3,3,1], strides=[1,2,2,1], padding="SAME")  
  # 14 14 conv1_fmaps
  # NONE , 14, 14, 32
  print("pool1 shape ", pool1.shape)
  
  
  conv2 = tf.layers.conv2d(pool1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.selu, name="conv2")
  # NONE , 14, 14, 64
  print("conv2 shape  ", conv2.shape)
  
  
  pool2 = tf.nn.max_pool(conv2, ksize=[1,3,3,1], strides=[1,2,2,1], padding="SAME")
  
  # NONE , 7, 7,64
  print("pool2 shape ", pool2.shape)

  
  conv3_fmaps = 128
  conv3_ksize = 3
  conv3_stride = 1
  conv3_pad = "SAME"
  
  conv3 = tf.layers.conv2d(pool2, filters=conv3_fmaps, kernel_size=conv3_ksize,
                         strides=conv3_stride, padding=conv3_pad,
                         activation=tf.nn.selu, name="conv3")

  
  print("conv3 shape ", conv3.shape)
  
  flat = tf.reshape(conv3, shape=[-1, conv3_fmaps * 7 * 7])
  print("flat shape ", flat.shape)


training = tf.placeholder_with_default(False, shape=(), name='training')
drop_out = 0.5

n_fc1 = 128
# n_fc2 = 128

with tf.name_scope("fc"):
#   fc1 = tf.layers.dense(flat , n_fc1, activation=tf.nn.selu, name="fc1")
#   fc2 = tf.layers.dense(fc1, n_fc2, activation=tf.nn.selu, name="fc2")
  
  dnn_inputs1 = tf.layers.dropout(flat, drop_out , training=training)
  fc1 = tf.layers.dense(dnn_inputs1 , n_fc1, activation=tf.nn.selu, name="fc1")
#   bn1 = tf.layers.batch_normalization(fc1, momentum = 0.95, training = training)
  
#   fc1_output = tf.nn.selu(bn1)

  
#   dnn_inputs2 = tf.layers.dropout(fc1_output, drop_out , training=training)
#   fc2 = tf.layers.dense(dnn_inputs2, n_fc2, activation=tf.nn.selu, name="fc2")
#   bn2 = tf.layers.batch_normalization(fc2, momentum = 0.95, training = training)

  dnn_output = fc1

  
with tf.name_scope("output"):
    logits = tf.layers.dense(dnn_output, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")


with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)    
    loss_summary = tf.summary.scalar("loss_summary", loss)

    
with tf.name_scope("training"):
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    acc_summary = tf.summary.scalar("accuracy", accuracy)    

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

인풋 shape  (?, 28, 28, 1)
conv1 shape  (?, 28, 28, 32)
pool1 shape  (?, 14, 14, 32)
conv2 shape   (?, 14, 14, 64)
pool2 shape  (?, 7, 7, 64)
conv3 shape  (?, 7, 7, 128)
flat shape  (?, 6272)


##Mnist Data 준비(Train /Valid /Test ) 

In [41]:

from tensorflow.examples.tutorials.mnist import input_data

tf.logging.set_verbosity(tf.logging.ERROR)  # deprecated 경고 메세지를 출력하지 않기 위해
mnist = input_data.read_data_sets("./tmp/data/")
tf.logging.set_verbosity(tf.logging.INFO)

X_train = mnist.train.images
y_train = mnist.train.labels

X_valid = mnist.validation.images
y_valid = mnist.validation.labels

X_test = mnist.test.images
y_test = mnist.test.labels


Extracting ./tmp/data/train-images-idx3-ubyte.gz
Extracting ./tmp/data/train-labels-idx1-ubyte.gz
Extracting ./tmp/data/t10k-images-idx3-ubyte.gz
Extracting ./tmp/data/t10k-labels-idx1-ubyte.gz


## Mnist label 준비



In [0]:

# 훈련 세트로부터 n_inputs와 n_outputs를 구하기
# 기존 label 값에 idx 를 붙이는거. 그러니까 0` 9 가 아닌  다른값일 때 값 정돈하는 방법
classes = np.unique(y_train)
# n_outputs = len(classes)

class_to_index = {
    label: index for index, label in enumerate(classes)
}
print(class_to_index)

## Train 
  - epoch : 100
  - batch_size : 128

### 조기 종료를 위한 var

- max_checks_without_progress = 20
- checks_without_progress = 0
- best_loss = np.infty
- best_params = None


### Param 저장 및 복원

- get_model_params()
- set_model_params()

In [0]:
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

def restore_model_params(model_params):
    gvar_names = list(model_params.keys())
    assign_ops = {
        gvar_name: tf.get_default_graph().get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    tf.get_default_session().run(assign_ops, feed_dict=feed_dict)

### Mnist Fitting

In [88]:
tf.summary.merge_all()
from datetime import datetime
# epochs 와 batch_Szie 
epochs = 100
batch_size =128

#best_param
max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty
best_params = None



def log_dir(output_path, prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    name = prefix + "run-" + now
    return "{}/{}/".format(output_path, name)
  
file_writer = tf.summary.FileWriter(log_dir("board_log"), graph=tf.get_default_graph() )



with tf.Session() as sess:
  sess.run(init)
  
  for epoch in range(epochs):
    rnd_idx = np.random.permutation(len(X_train))
    for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
      X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
      
      feed_dict = {X : X_batch, y: y_batch}
      if training is not None:
        feed_dict[training] = True                    
      
      sess.run(training_op, feed_dict= feed_dict)
    
#       if extra_update_ops:
#           sess.run(extra_update_ops, feed_dict=feed_dict)
          
    if epoch % 5 ==0 :
      acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})

      loss_val, acc_val, loss_smm, acc_smm = sess.run([loss, accuracy, loss_summary, acc_summary], 
                                                      feed_dict={X: X_valid, y: y_valid})
      
      if loss_val < best_loss:
#         gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
#         best_params = {gvar.op.name: value for gvar, value in zip(gvars, sess.run(gvars))}
        best_loss = loss_val
        best_params= get_model_params()
        checks_without_progress = 0
      else:
        checks_without_progress +=1 

      print(epoch, "훈련 정확도:", acc_train, "검증 세트 정확도:", acc_val, "검증 loss val: ", loss_val)
      file_writer.add_summary(summary=acc_smm, global_step=epoch)
      file_writer.add_summary(summary=loss_smm, global_step=epoch)
      if checks_without_progress > max_checks_without_progress:
        print("조기 종료!")
        break

  
  if best_params:  
    restore_model_params(best_params)
  acc_test = accuracy.eval(feed_dict={X: X_test,
                                        y: y_test})
  print("테스트 세트에서 최종 정확도:", acc_test)

  



0 훈련 정확도: 0.9765625 검증 세트 정확도: 0.9848 검증 loss val:  0.05518033
5 훈련 정확도: 1.0 검증 세트 정확도: 0.9914 검증 loss val:  0.038408432
10 훈련 정확도: 1.0 검증 세트 정확도: 0.9874 검증 loss val:  0.058705695
15 훈련 정확도: 1.0 검증 세트 정확도: 0.9936 검증 loss val:  0.04517377
20 훈련 정확도: 1.0 검증 세트 정확도: 0.9906 검증 loss val:  0.06433026
25 훈련 정확도: 0.9921875 검증 세트 정확도: 0.9904 검증 loss val:  0.07314846
30 훈련 정확도: 1.0 검증 세트 정확도: 0.9918 검증 loss val:  0.07399049
35 훈련 정확도: 1.0 검증 세트 정확도: 0.9908 검증 loss val:  0.11311185
40 훈련 정확도: 1.0 검증 세트 정확도: 0.992 검증 loss val:  0.12022979
45 훈련 정확도: 1.0 검증 세트 정확도: 0.9902 검증 loss val:  0.15862979
50 훈련 정확도: 1.0 검증 세트 정확도: 0.9908 검증 loss val:  0.13868095
55 훈련 정확도: 1.0 검증 세트 정확도: 0.9942 검증 loss val:  0.09519664
60 훈련 정확도: 1.0 검증 세트 정확도: 0.992 검증 loss val:  0.13435438
65 훈련 정확도: 1.0 검증 세트 정확도: 0.9932 검증 loss val:  0.18239403
70 훈련 정확도: 1.0 검증 세트 정확도: 0.9936 검증 loss val:  0.13058555
75 훈련 정확도: 1.0 검증 세트 정확도: 0.9928 검증 loss val:  0.17161426
80 훈련 정확도: 1.0 검증 세트 정확도: 0.993 검증 loss val:  0.22695093
85 훈련